In [21]:
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
import sys
sys.path.append('../../')
from bert import nli
import tensorflow as tf
import re

In [48]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)

nli_labels = ["contradiction", "entailment", "neutral"]
rumor_labels = ["rumor", "truth", "unconfirmed claim","nonrumor"]

In [51]:
tf.logging.set_verbosity(tf.logging.ERROR)
predict_fn = nli.get_nli_predict_fn(
    bert_base_dir = '../../chinese-bert_chinese_wwm_L-12_H-768_A-12',
    output_dir = '../../xnli_output'
)

In [3]:
model = SentenceTransformer('distiluse-base-multilingual-cased')

/home/link/miniconda3/envs/bert/lib/python3.6/site-packages/transformers/tokenization_utils.py:831: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  category=FutureWarning,


In [4]:
dxy_rumors=pd.read_csv('../data/DXY_rumors.csv')
dxy_rumors = dxy_rumors.drop(['_id','id','mainSummary','summary','sourceUrl','crawlTime','body'],axis=1)

In [75]:
def detect_rumor(input_text):
    text_b = input_text
    x = np.array(model.encode([text_b]))
    z = np.array(model.encode(dxy_rumors.title.values))
    f=z.dot(x.transpose()).reshape(284)
    p=softmax(f)
    most_match_index = np.argmax(p)
    text_a = re.sub('[？?]','',dxy_rumors.title.values[most_match_index])
    label_a = dxy_rumors.rumorType.values[most_match_index]
    #dxy_rumors.values[np.where(p>0.004)]
    #print(text_a,rumor_labels[label_a])
    examples = [
        nli.InputExample(guid='prodiction-1',text_a=text_a,text_b=text_b,label='neutral')
    ]
    result = predict_fn(examples)
    for (i, prediction) in enumerate(result):
        probabilities = prediction['probabilities']
        print(text_a,'->',text_b,': ',nli_labels[np.argmax(probabilities)])
        if np.argmax(probabilities)==0:
            rumor_labels = ["truth", "rumor", "unconfirmed claim","nonrumor"]
        else:
            rumor_labels = ["rumor", "truth", "unconfirmed claim","nonrumor"]
        if np.argmax(probabilities)<2:
            print(text_b,' is ',rumor_labels[label_a])
        else:
            print('Most related '+rumor_labels[label_a]+' is: ',text_a)


In [63]:
detect_rumor('钟南山没被传染')

钟南山院士被传染 -> 钟南山没被传染 :  contradiction
钟南山没被传染  is  truth


In [64]:
detect_rumor('钟南山被传染')

钟南山院士被传染 -> 钟南山被传染 :  entailment
钟南山被传染  is  rumor


In [65]:
detect_rumor('奥司他韦可以用于预防新冠病毒')

吃奥司他韦可以预防新冠病毒 -> 奥司他韦可以用于预防新冠病毒 :  entailment
奥司他韦可以用于预防新冠病毒  is  rumor


In [69]:
detect_rumor('吃双黄连不可以预防病毒')

双黄连可以预防新冠病毒 -> 吃双黄连不可以预防病毒 :  contradiction
吃双黄连不可以预防病毒  is  truth


In [78]:
detect_rumor('吃双黄连可以预防病毒')

双黄连可以预防新冠病毒 -> 吃双黄连可以预防病毒 :  entailment
吃双黄连可以预防病毒  is  rumor


In [82]:
detect_rumor('汉堡好吃')

家畜家禽海鲜都不能吃了 -> 汉堡好吃 :  neutral
Most related rumor is:  家畜家禽海鲜都不能吃了
